In [ ]:
import pandas as pd
import pathlib 

kWh_per_gal = 33.41 # https://en.wikipedia.org/wiki/Gasoline_gallon_equivalent
km_per_mile = 1.609344

In [ ]:
# From https://www.fueleconomy.gov/feg/download.shtml


def process_fuel_economy(file):

    year = int("20" + file.name[10:12])
    df = pd.read_excel(file)

    mask = df["Cert Region"] == "FA"
    mask &= df["Fuel"] != "Gasoline/Electricity"
    mask &= df["Fuel"] != "Ethanol/Gas"
    mask &= df["Fuel"] != "Electricity/Hydrogen"
    mask &= df["Fuel"] != "Hydrogen"
    mask &= df["Fuel"] != "Diesel"
    df = df[mask].reset_index()

    mapper = {"City MPG": "float64", "Hwy MPG": "float64", "Cmb MPG": "float64"}
    for col, dtype in mapper.items():
        df[col] = df[col].astype(dtype)

    mask = ~df.Displ.isna()
    cyl_mask = ~df.Cyl.isna()
    df.loc[mask, "ID"] = (
        df.loc[mask, "Model"] + " " + df.loc[mask, "Displ"].astype(str) + "L"
    )
    df.loc[cyl_mask, "ID"] = (
        df.loc[cyl_mask, "ID"]
        + " "
        + df.loc[cyl_mask, "Cyl"].astype(int).astype(str)
        + " Cylinder"
    )
    df["ID"] = df["ID"].fillna(df.Model)
    df["ID"] += " " + df["Drive"]
    df["ID"] += " " + df["Trans"]

    df["Year"] = year
    df["ID"] = df["Year"].astype(str) + " " + df["ID"]

    df["City kWh/km"] = kWh_per_gal / (df["City MPG"] * km_per_mile)
    df["Hwy kWh/km"] = kWh_per_gal / (df["Hwy MPG"] * km_per_mile)
    df["Cmb kWh/km"] = kWh_per_gal / (df["Cmb MPG"] * km_per_mile)

    models = df.groupby(["ID", "Year", "Fuel", "Veh Class"]).agg(
        {"City kWh/km": "mean", "Hwy kWh/km": "mean", "Cmb kWh/km": "mean"}
    )
    namer = {"City kWh/km": "City", "Hwy kWh/km": "Highway", "Cmb kWh/km": "Combined"}
    models = models.rename(columns=namer).reset_index()

    return models

In [ ]:
models = []
for file in pathlib.Path().glob('all_alpha*.xlsx'):
    year_models = process_fuel_economy(file)
    models.append(year_models)

models = pd.concat(models,axis=0)
models.to_csv('models.csv',index=False)